### Load Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Check Drive

In [ ]:
ls /content/drive/MyDrive/model

### Install Packages

In [ ]:
!pip install -q torch torchvision opencv-python flask flask-cors got10k pyngrok

### Copy from Drive to Colab

In [ ]:
!cp -r /content/drive/MyDrive/model /content/

### Check

In [ ]:
print("\nModel files:")
!ls -lh /content/model/

In [ ]:
import os

required_files = ['model.pth', 'siamrpn.py', 'colab_api.py']
model_dir = '/content/model'

print("Checking required files...\n")
all_present = True
for file in required_files:
    filepath = os.path.join(model_dir, file)
    if os.path.exists(filepath):
        size = os.path.getsize(filepath) / (1024 * 1024)  # MB
        print(f"✓ {file} ({size:.2f} MB)")
    else:
        print(f"✗ {file} - MISSING!")
        all_present = False

if all_present:
    print("\n✓ All files present! Ready to continue.")
else:
    print("\n⚠️  Some files are missing. Please upload them before continuing.")

### Connect ngrok

In [ ]:
from pyngrok import ngrok
import os

# OPTIONAL:
# Get free token from: https://dashboard.ngrok.com/authtokens
ngrok.set_auth_token("3672mGsbiY65MtOFmbDIOII3xoU_WDqAxVpX459DMFmmRxtt")

ngrok.kill()

public_url = ngrok.connect(5000, bind_tls=True)

print("\n" + "=" * 70)
print("API IS READY!")
print("=" * 70)
print(f"\nPUBLIC URL: {public_url}")
print(f"\nEndpoints:")
print(f"   Health Check: {public_url}/health")
print(f"   Track Video:  {public_url}/track")
print("\n" + "=" * 70)
print("IMPORTANT:")
print("   1. Copy the PUBLIC URL above")
print("   2. Paste it in your Next.js .env.local file:")
print(f"      NEXT_PUBLIC_API_URL={public_url}")
print("   3. Keep this notebook running!")
print("=" * 70 + "\n")

# Save URL to file for easy access
with open('/content/api_url.txt', 'w') as f:
    f.write(str(public_url))
print("✓ URL saved to /content/api_url.txt")

### Start the Flask API server

In [ ]:
%cd /content/model
!python colab_api.py

### Test

In [ ]:
import requests

with open('/content/api_url.txt', 'r') as f:
    api_url = f.read().strip()

try:
    response = requests.get(f"{api_url}/health")
    print("Health Check Response:")
    print(response.json())
    print("\n✓ API is working!")
except Exception as e:
    print(f"✗ Error: {e}")
    print("Make sure the API server is running (Step 4)")